# Hugging face eval cnn-dm
+ 使用 cnn dailymail 通过 Hugging Face 的各个模型效果评估
+ https://github.com/hellotransformers/Natural_Language_Processing_with_Transformers
+ https://github.com/hellotransformers/Natural_Language_Processing_with_Transformers/blob/main/chapter6.md
+ https://xiaosheng.run/2022/03/29/transformers-note-8.html
+ https://github.com/datawhalechina/learn-nlp-with-transformers/blob/main/docs/%E7%AF%87%E7%AB%A04-%E4%BD%BF%E7%94%A8Transformers%E8%A7%A3%E5%86%B3NLP%E4%BB%BB%E5%8A%A1/4.7-%E7%94%9F%E6%88%90%E4%BB%BB%E5%8A%A1-%E6%91%98%E8%A6%81%E7%94%9F%E6%88%90.md

In [1]:
# 使用终端安装完包之后要重启jupyter
!which python
# !pip list
!export http_proxy="http://172.19.57.45:3128/"
!export https_proxy="http://172.19.57.45:3128/"
# !export TRANSFORMERS_OFFLINE=1 # 强制离线加载模型
# # !export TRANSFORMERS_OFFLINE=0 # 强制在线
# !export HF_DATASETS_OFFLINE=1 # 强制离线加载数据
# !export http_proxy=''
# !export http_proxy=''
# dataset 2.5.2
# !pip install datasets==2.5.2
# !pip install sacrebleu==2.3.1
# rouge_score==0.0.4 work well



/home/users/sunhongchao/miniconda3/envs/bot-mvp-3/bin/python


# 测试gpu使用情况

In [2]:
import torch
torch.cuda.is_available()

/home/users/sunhongchao/miniconda3/envs/bot-mvp-3/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# CNN DM 数据介绍
+ CNN/DailyMail数据集由大约300,000对新闻文章及其相应的摘要组成，这些摘要由CNN和DailyMail在其文章中附加的要点组成
+ 该数据集的一个重要方面是，摘要是抽象的，而不是摘录的，这意味着它们由新的句子而不是简单的摘录组成
+ 该数据集可在Hub上找到；我们将使用3.0.0版本，这是一个为摘要而设置的非匿名版本
+ 训练集大小： 286817
+ 验证集大小： 13368
+ 测试集大小： 11487
+ 训练集中平均摘要句子数： 3.72

# 数据准备
+ 因为网络或者代理的问题，数据从云端直接下载有问题，解决方案如下
+ 远程加载 可以参考 https://github.com/huggingface/datasets/issues/996
+ 本地加载 可以参考 https://blog.csdn.net/PolarisRisingWar/article/details/124042709

In [3]:
# 代理必须关闭
# 服务器上也需要关闭代理
# hide_output
import datasets
from datasets import load_dataset

# 远程加载
# dataset = load_dataset("cnn_dailymail",  version="3.0.0") # 有bug
# dataset = load_dataset("ccdv/cnn_dailymail",  version="3.0.0")
# 本地加载
dataset = datasets.load_from_disk('hf_cnn-dm')
print(f"Features: {dataset['train'].column_names}")


Features: ['article', 'highlights', 'id']


该数据集有三列：文章，其中包含新闻文章，亮点与摘要，以及唯一标识每篇文章的ID

In [4]:
sample = dataset["train"][1]
print(f"""
>>> Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\n>>> Summary (length: {len(sample["highlights"])}):')
print(sample["highlights"])


>>> Article (excerpt of 500 characters, total length: 3192):

(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has n

>>> Summary (length: 180):
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .


In [5]:
sample_text = dataset["train"][1]["article"][:2000]
print(sample_text)
# We'll collect the generated summaries of each model in a dictionary
summaries = {}

(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory was never seriou

In [6]:
import nltk
from nltk.tokenize import sent_tokenize

from transformers import set_seed 
from transformers import pipeline
import torch
set_seed(42) 


In [7]:
# # use bart in pytorch
# from transformers import  pipeline
# summarizer = pipeline("summarization",proxy='')
# summarizer("An apple a day, keeps the doctor away", min_length=5, max_length=20)

# # use t5 in tf
# summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")
# summarizer("An apple a day, keeps the doctor away", min_length=5, max_length=20)

# baseline

In [8]:
def three_sentence_summary(text): 
	return "\n".join(sent_tokenize(text)[:3]) 
summaries["baseline"] = three_sentence_summary(sample_text)

# gpt2

In [9]:

pipe = pipeline("text-generation", model="../../../resources/embedding/gpt2-xl")
# pipe = pipeline("text-generation", model="gpt2-xl")
gpt2_query = sample_text + "\nTL;DR:\n" 
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)

summaries["gpt2"] = "\n".join( sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))
print(summaries["gpt2"])

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


The Olympic champion wins gold in Moscow.
Bolt gets back in the picture with an impressive silver.
A triple by Fraser-Pryce, Bailey-Cole, and Carter leaves the Jamaicans in the lead for the last part of the relay.
Bolt dominates the final 50 meters and finishes off his hat trick and completes his double by completing the triple on the home straight, winning the gold with a world-


# t5

In [10]:
pipe = pipeline("summarization", model="../../../resources/embedding/t5-large") 
pipe_out = pipe(sample_text) 
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
print(summaries["t5"] )

/home/users/sunhongchao/miniconda3/envs/bot-mvp-3/lib/python3.6/site-packages/transformers/generation_utils.py:745: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


usain bolt wins his third gold of the world championships in the men's 4x100m relay .
the 26-year-old anchored the Jamaican quartet to victory in 37.36 seconds .
he has now collected eight gold medals at the championships, equaling the record .


# bart

In [11]:
pipe = pipeline("summarization", model="../../../resources/embedding/facebook_bart-large-cnn") 
pipe_out = pipe(sample_text) 
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
print(summaries["bart"])

Usain Bolt wins his third gold of the world championships in Moscow.
Bolt anchors Jamaica to victory in the men's 4x100m relay.
The 26-year-old has now won eight gold medals at world championships.
Jamaica's women also win gold in the relay, beating France in the process.


# pegasus

In [12]:
import torch
with torch.no_grad():
    # pipe = pipeline("summarization", model="../../../resources/embedding/google_pegasus-cnn-dailymail") 
    pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail") 
    pipe_out = pipe(sample_text) 
    summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")
print(summaries["pegasus"])

Usain Bolt wins third gold of world championships.
Anchors Jamaica to victory in men's 4x100m relay.
Eighth gold at the championships for Bolt.
Jamaica also win women's 4x100m relay .


# 不同模型效果对比

In [13]:
print("GROUND TRUTH") 
print(dataset["train"][1]["highlights"]) 
print("") 
for model_name in summaries: 
	print(model_name.upper()) 
	print(summaries[model_name]) 
	print("")

GROUND TRUTH
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .

BASELINE
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.
The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.
The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover.

GPT2
The Olympic champion wins gold in Moscow.
Bolt gets back in the picture with an impressive silver.
A triple by Fraser-Pryce, Bailey-Cole, and Carter leaves the Jamaicans in the lead for the last part of the relay.
Bolt dominates the final 50 meters and finishes off his hat trick and completes his double by completi

# 评估指标

In [14]:
from datasets import load_metric 
bleu_metric = load_metric("sacrebleu") 

Using the latest cached version of the module from /home/users/sunhongchao/.cache/huggingface/modules/datasets_modules/metrics/sacrebleu/31e1673407d8789b8f5ddfd979948f6a1de0a6d691426d55fa74a35ffb0c1bdf (last modified on Tue Oct 25 00:02:35 2022) since it couldn't be found locally at sacrebleu, or remotely on the Hugging Face Hub.


In [15]:
import pandas as pd 
import numpy as np 
bleu_metric.add( prediction="the the the the the the", reference=["the cat is on the mat"]) 
results = bleu_metric.compute(smooth_method="floor", smooth_value=0) 
results["precisions"] = [np.round(p, 2) for p in results["precisions"]] 
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1
sys_len,6
ref_len,6


In [16]:
bleu_metric.add( prediction="the cat is on mat", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0) 
results["precisions"] = [np.round(p, 2)for p in results["precisions"]] 
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,57.893
counts,"[5, 3, 2, 1]"
totals,"[5, 4, 3, 2]"
precisions,"[100.0, 75.0, 66.67, 50.0]"
bp,0.818731
sys_len,5
ref_len,6


In [17]:
# rouge_score==0.0.4 work well
rouge_metric = load_metric("rouge")

Using the latest cached version of the module from /home/users/sunhongchao/.cache/huggingface/modules/datasets_modules/metrics/rouge/0ffdb60f436bdb8884d5e4d608d53dbe108e82dac4f494a66f80ef3f647c104f (last modified on Tue Oct 25 19:02:08 2022) since it couldn't be found locally at rouge, or remotely on the Hugging Face Hub.


In [18]:
reference = dataset["train"][1]["highlights"] 
records = [] 
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"] 
for model_name in summaries: 
	rouge_metric.add(prediction=summaries[model_name], reference=reference) 
	score = rouge_metric.compute() 
	rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
	records.append(rouge_dict) 
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.303571,0.090909,0.214286,0.232143
gpt2,0.244898,0.000000,0.122449,0.224490
t5,0.472222,0.228571,0.388889,0.472222
bart,0.582278,0.207792,0.455696,0.506329
pegasus,0.866667,0.655172,0.800000,0.833333


# 使用pegasus 抽样评估 测试集


In [19]:
def evaluate_summaries_baseline(dataset, metric, column_text="article", column_summary="highlights"): 
    summaries = [three_sentence_summary(text) for text in dataset[column_text]] 
    metric.add_batch(predictions=summaries, references=dataset[column_summary]) 
    score = metric.compute() 
    return score

In [20]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(1000)) 
score = evaluate_summaries_baseline(test_sampled, rouge_metric) 
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names) 
pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

Loading cached shuffled indices for dataset at hf_cnn-dm/test/cache-f551f6b1b06308aa.arrow


,rouge1,rouge2,rougeL,rougeLsum
baseline,0.388071,0.170554,0.247146,0.354972


In [21]:
from tqdm import tqdm
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu" 
def chunks(list_of_elements, batch_size): 
	"""Yield successive batch-sized chunks from list_of_elements.""" 
	for i in range(0, len(list_of_elements), batch_size): 
	    yield list_of_elements[i : i + batch_size] 
def evaluate_summaries_pegasus(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text="article", column_summary="highlights"): 
	article_batches = list(chunks(dataset[column_text], batch_size)) 
	target_batches = list(chunks(dataset[column_summary], batch_size)) 
	for article_batch, target_batch in tqdm( zip(article_batches, target_batches), total=len(article_batches)): 
		inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 
		summaries = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device), length_penalty=0.8, num_beams=8, max_length=128) 
		decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries] 
		decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries] 
		metric.add_batch(predictions=decoded_summaries, references=target_batch) 
	score = metric.compute() 
	return score

In [22]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer 

# import torch
# with torch.no_grad():
#     model_ckpt = "google/pegasus-cnn_dailymail" 
#     tokenizer = AutoTokenizer.from_pretrained(model_ckpt) 
#     model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device) 
#     score = evaluate_summaries_pegasus(test_sampled, rouge_metric, model, tokenizer, batch_size=8) 
#     rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names) 
#     pd.DataFrame(rouge_dict, index=["pegasus"])

# print(rouge_dict)

In [23]:
print(rouge_dict)

{'rouge1': 0.38807104712508933, 'rouge2': 0.17055379457597958, 'rougeL': 0.2471461969561793, 'rougeLsum': 0.35497217427274075}


# 训练一个摘要模型
+ 使用 SAMSum
+ SAMSum 数据介绍如下

In [24]:
# dataset_samsum = load_dataset("samsum") 
dataset_samsum = datasets.load_from_disk('hf_samsum')
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum] 
print(f"Split lengths: {split_lengths}") 
print(f"Features: {dataset_samsum['train'].column_names}") 
print("\nDialogue:") 
print(dataset_samsum["test"][0]["dialogue"]) 
print("\nSummary:") 
print(dataset_samsum["test"][0]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


# 流水线评估

In [25]:
pipe_out = pipe(dataset_samsum["test"][0]["dialogue"]) 
print("Summary:") 
print(pipe_out[0]["summary_text"].replace(" .<n>", ".\n")) 

Your max_length is set to 128, but you input_length is only 122. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
/home/users/sunhongchao/miniconda3/envs/bot-mvp-3/lib/python3.6/site-packages/transformers/generation_utils.py:745: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together.
Hannah: I'd rather you texted him.
Amanda: Just text him .


我们可以看到，该模型大多试图通过提取对话中的关键句子来进行文本摘要。这在CNN/DailyMail数据集上可能效果相对较好，但SAMSum中的文本摘要更加抽象。让我们通过在测试集上运行完整的ROUGE评估来确认这一点:

In [26]:
score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, model, tokenizer, column_text="dialogue", column_summary="summary", batch_size=8) 

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names) 
pd.DataFrame(rouge_dict, index=["pegasus"])

NameError: name 'model' is not defined

结果不是很好，但这并不意外，因为我们已经远离了CNN/DailyMail的数据分布。尽管如此，在训练前设置评估流水线有两个好处：我们可以直接用指标来衡量训练的成功与否，而且我们有一个好的基线。在我们的数据集上对模型进行微调，应该会使ROUGE指标立即得到改善，如果不是这样，我们就知道我们的训练循环出了问题。

# 微调 pegasus


In [ ]:
import matplotlib.pyplot as plt
d_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"] ["dialogue"]] 
s_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["summary"]] 
fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
axes[0].hist(d_len, bins=20, color="C0", edgecolor="C0") 
axes[0].set_title("Dialogue Token Length") 
axes[0].set_xlabel("Length") 
axes[0].set_ylabel("Count") 
axes[1].hist(s_len, bins=20, color="C0", edgecolor="C0") 
axes[1].set_title("Summary Token Length") 
axes[1].set_xlabel("Length") 
plt.tight_layout() 

plt.show()


我们看到，大多数对话比CNN/DailyMail的文章短得多，每个对话有100-200个标记。同样，摘要也短得多，大约有20-40个符号（一条推文的平均长度）
让我们在为训练者建立数据整理器时牢记这些意见。首先，我们需要对数据集进行标记。现在，我们将对话和摘要的最大长度分别设置为1024和128:

In [ ]:
# def convert_examples_to_features(example_batch): 
# 	input_encodings = tokenizer(example_batch["dialogue"], max_length=1024, truncation=True) 
# 	with tokenizer.as_target_tokenizer(): 
# 		target_encodings = tokenizer(example_batch["summary"], max_length=128, truncation=True) 
# 	return {"input_ids": input_encodings["input_ids"], "attention_mask": input_encodings["attention_mask"], "labels": target_encodings["input_ids"]} 
# dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True) 
# columns = ["input_ids", "labels", "attention_mask"] 
# dataset_samsum_pt.set_format(type="torch", columns=columns)

def convert_examples_to_features(example_batch): 
	input_encodings = tokenizer(example_batch["dialogue"], max_length=1024, truncation=True) 
	# with tokenizer.as_target_tokenizer(): 
	target_encodings = tokenizer(example_batch["summary"], max_length=128, truncation=True) 
	return {"input_ids": input_encodings["input_ids"], "attention_mask": input_encodings["attention_mask"], "labels": target_encodings["input_ids"]} 
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True) 
columns = ["input_ids", "labels", "attention_mask"] 
dataset_samsum_pt.set_format(type="torch", columns=columns)

使用标记化步骤的一个新东西是tokenizer.as_target_tokenizer()上下文。有些模型在解码器输入中需要特殊的标记，所以区分编码器和解码器输入的标记很重要。在with语句（称为上下文管理器）中，标记器知道它正在为解码器进行标记，并可以相应地处理序列。

In [ ]:
from transformers import DataCollatorForSeq2Seq 
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) 

#然后，像往常一样，我们为训练设置了一个TrainingArguments:

from transformers import TrainingArguments, Trainer 
training_args = TrainingArguments( output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500, per_device_train_batch_size=1, per_device_eval_batch_size=1, weight_decay=0.01, logging_steps=10, push_to_hub=True,
evaluation_strategy='steps', eval_steps=500, save_steps=1e6, gradient_accumulation_steps=16)

In [ ]:
trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer, data_collator=seq2seq_data_collator, train_dataset=dataset_samsum_pt["train"], eval_dataset=dataset_samsum_pt["validation"])


In [ ]:
trainer.train() 
score = evaluate_summaries_pegasus( dataset_samsum["test"], rouge_metric, trainer.model, tokenizer, batch_size=2, column_text="dialogue", column_summary="summary") 
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names) 
pd.DataFrame(rouge_dict, index=[f"pegasus"])
